In [6]:
example = """
939
7,13,x,x,59,x,31,19
""".strip().splitlines()

with open('day13.txt', 'r') as f:
    data = f.readlines()

## Part 1 Solution
To solve **Part 1** we can take advantage of the fact that each bus arrives at the same period. This useful property
allows us to use simple multiplication to determine all of the times that a bus will arrive at.

```
t_arrival = n_arrival * bus_id
          = 0 * bus_id
          = 1 * bus_id
          = 2 * bus_id
          = ...
```

So given that our arrival numbers are integers (i.e. the bus cannot half-arrive), we can determine the next time that
it is due by calculating the next arrival number and multiplying by the bus ID. The last arrival number is, coincidentally,
the `floor()` of the current timestamp divided by the bus ID (the length of its journey).

So in other words, `t_arrival = (floor(time / bus_id) + 1) * bus_id`.

## Part 2 Solution
The solution to **Part 2** is a bit more complicated, but it generally takes advantage of the fact that these busses arrive
in predictable periods. If we take the first bus (in the example) it will arrive every 7 minutes. To maintain the same cycle
between the first and second busses, we can use the Lowest Common Multiple (`7 * 13`) to determine when the cycle between the
two busses will repeat.

Thus, if we wished to find the cycle-length in which all busses arrived at the same time, we would simply need to multiply
all of the bus IDs together (e.g. `7 * 13 * 59 * 31 * 19`).

However, we also wish to find a certain offset from the busses all arriving at identical times. One way to achieve this is to
cycle each bus in turn until we arrive at the desired offset and then "freeze" that portion of the cycle.

For example, if `7` starts at the desired offset (`0`) and we wish to offset `13` by `1` relative to `7` then we need to cycle
by `7` each step until `13` arrives one minute after bys `7`. We can then freeze this pair by cycling in increments of `7 * 13`
and so on.

In [20]:
from typing import List, Dict

class BusSchedule(object):
    def __init__(self, buses: List[str]):
        self.busses = [
            int(bus.strip())
            for bus in buses
            if bus != 'x'
        ]
        self.indexes = [
            i
            for i, bus in enumerate(buses)
            if bus != 'x'
        ]

    def get_next_time(self, bus: int, after: int) -> int:
        return (int(after/bus) + 1) * bus

    def get_next_bus(self, timestamp: int):
        min_bus = self.busses[0]
        for bus in self.busses:
            if self.get_next_time(bus, timestamp) < self.get_next_time(min_bus, timestamp):
                min_bus = bus

        return min_bus

    def get_part1_answer(self, timestamp: int) -> int:
        next_bus = self.get_next_bus(timestamp)
        time_to_wait = self.get_next_time(next_bus, timestamp) - timestamp
        return next_bus * time_to_wait

    def get_part2_answer(self) -> int:
        offsets = [
            -i % bus for bus, i in zip(self.busses, self.indexes)
        ]

        timestamp = 0
        step = 1
        for bus, offset in zip(self.busses, offsets):
            while timestamp % bus != offset:
                timestamp += step
            step *= bus

        return timestamp


example_schedule = BusSchedule(example[1].split(','))
print(f"Next bus: {example_schedule.get_next_bus(int(example[0].strip()))}")
print(f"Part 1: {example_schedule.get_part1_answer(int(example[0].strip()))}")
print(f"Part 2: {example_schedule.get_part2_answer()}")

Next bus: 59
Part 1: 295
Part 2: 1068781


In [21]:
true_schedule = BusSchedule(data[1].split(','))
print(f"Part 1: {true_schedule.get_part1_answer(int(data[0].strip()))}")
print(f"Part 2: {true_schedule.get_part2_answer()}")

Part 1: 174
Part 2: 780601154795940
